In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

## Modelos 2

Uma continuação.

Agora inicialmente tentaremos verificar se a inclusão de MTRANS com um Colum transform e removendo 

In [3]:
df =pd.read_pickle('df_transformado.pkl')
df_test = pd.read_pickle('df_test_transformado.pkl')

df_o = pd.read_csv('../../playground-series-s4e2/train.csv')
df_test_o = pd.read_csv("../../playground-series-s4e2/test.csv")


In [ ]:
test